## Llama모델 학습
### 사용환경: Colab

In [ ]:
!pip install pyfiglet
from pyfiglet import Figlet
f = Figlet(font='slant')
print(f.renderText('HELLO SILVERPRIZE'))

### 키 파일 생성

In [ ]:

import json
from google.colab import userdata

data = {
  "type": "service_account",
  "project_id": "llama-433214",
  "private_key_id": userdata.get("private_key_id"),
  "private_key": userdata.get("private_key").replace("\\n", "\n"),
  "client_email": "storage-manager@llama-433214.iam.gserviceaccount.com",
  "client_id": "112540232109707769744",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/storage-manager%40llama-433214.iam.gserviceaccount.com",
  "universe_domain": "googleapis.com"
}

json_file_path = "key.json"
with open(json_file_path, "w") as json_file:
    json.dump(data, json_file)

### Google Auth

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./key.json"

### 모델 다운로드

In [ ]:
from google.cloud import storage
bucket_name = "bucket-llamamodels"

# upload file
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

# download file
def download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    os.makedirs(os.path.dirname(destination_file_name), exist_ok=True)

    blob.download_to_filename(destination_file_name)

    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")


# downolad folder
def download_folder(bucket_name, source_folder_name, destination_folder_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    blobs = storage_client.list_blobs(bucket_name, prefix=source_folder_name)

    for blob in blobs:
        relative_path = os.path.relpath(blob.name, source_folder_name)
        local_file_path = os.path.join(destination_folder_name, relative_path)
        download_blob(bucket_name, blob.name, local_file_path)

In [ ]:
# 경로 지정
projectPath = os.getcwd() # 현재 위치, /content

# 버킷 내 다운로드할 위치
# https://console.cloud.google.com/storage/browser/bucket-llamamodels;tab=objects?forceOnBucketsSortingFiltering=true&project=llama-433214&prefix=&forceOnObjectsSortingFiltering=false
prefix = "gguf/"

# 다운로드할 폴더 이름
folderPath = "Llama-3.1-Korean-8B-Instruct/"

sourcePath = prefix + folderPath
destPath = os.path.join(projectPath, folderPath)

In [ ]:
# 다운로드
download_folder(bucket_name=bucket_name,
                source_folder_name=sourcePath,
                destination_folder_name=destPath)

---

## 모델 학습

In [ ]:
# Install Dependencies
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

In [ ]:
# Check GPU
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

In [ ]:
# 공용 변수들
